In [ ]:
import qutip as qt

from qubex.simulator.experimental.pulse_optimizer import PulseOptimizer
from qubex.simulator.experimental.quantum_simulator import (
    MultiControl,
    QuantumSimulator,
)
from qubex.simulator.experimental.quantum_system import (
    Coupling,
    QuantumSystem,
    Transmon,
)

In [ ]:
transmons = [
    Transmon(
        label="Q1",
        dimension=2,
        frequency=0.0,
        anharmonicity=-0.368,
    ),
    Transmon(
        label="Q2",
        dimension=2,
        frequency=1.0,
        anharmonicity=-0.500,
    ),
]

couplings = [
    Coupling(
        pair=("Q1", "Q2"),
        strength=0.01,
    ),
]

system = QuantumSystem(
    objects=transmons,
    couplings=couplings,
)

In [ ]:
system.hamiltonian

In [ ]:
I = qt.qeye(2)
X = qt.sigmax()
target_unitary = qt.tensor(X, I)
target_unitary

In [ ]:
segment_width = 4

optimizer = PulseOptimizer(
    quantum_system=system,
    target_unitary=target_unitary,
    initial_state=system.ground_state,
    control_frequencies={
        "Q1": transmons[0].frequency,
        "Q2": transmons[1].frequency,
    },
    segment_count=15,
    segment_width=segment_width,
    max_rabi_frequency=0.1,
)

In [ ]:
opt_result = optimizer.optimize(
    learning_rate=0.001,
    max_iterations=1000,
    tolerance=1e-6,
)

In [ ]:
control = MultiControl(
    frequencies={
        "Q1": transmons[0].frequency,
        "Q2": transmons[1].frequency,
    },
    waveforms=opt_result.waveforms,
    sampling_period=segment_width,
    steps_per_sample=10,
)

control.plot()

In [ ]:
simulator = QuantumSimulator(system)

In [ ]:
result = simulator.simulate(
    control=control,
    initial_state=system.ground_state,
)

In [ ]:
final_state = result.states[-1]
final_state

In [ ]:
final_state_target = result.substates("Q1")[-1]
final_state_target

In [ ]:
result.plot_population_dynamics()

In [ ]:
result.plot_population_dynamics("Q1")

In [ ]:
result.plot_population_dynamics("Q2")

In [ ]:
result.show_last_population("Q1")

In [ ]:
result.display_bloch_sphere("Q1")

In [ ]:
result.display_bloch_sphere("Q2")